https://github.com/oya163/bert-llm/blob/master/CyberSecurityNER/cyber_security_ner.ipynb

In [1]:
# !pip install -U huggingface_hub
!pip install accelerate
!pip install transformers
!install datasets evaluate
!pip install seqeval

Looking in indexes: https://artifactory.michelin.com/api/pypi/pypi/simple
Looking in indexes: https://artifactory.michelin.com/api/pypi/pypi/simple
install: cannot stat 'datasets': No such file or directory
Looking in indexes: https://artifactory.michelin.com/api/pypi/pypi/simple


In [2]:

!wget https://raw.githubusercontent.com/oya163/bert-llm/master/CyberSecurityNER/cyner_data/train.txt
!wget https://raw.githubusercontent.com/oya163/bert-llm/master/CyberSecurityNER/cyner_data/valid.txt
!wget https://raw.githubusercontent.com/oya163/bert-llm/master/CyberSecurityNER/cyner_data/test.txt
!wget https://raw.githubusercontent.com/oya163/bert-llm/master/CyberSecurityNER/load_ner.py

--2024-01-24 15:23:34--  https://raw.githubusercontent.com/oya163/bert-llm/master/CyberSecurityNER/cyner_data/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661664 (646K) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>] 646.16K  2.51MB/s    in 0.3s    

2024-01-24 15:23:35 (2.51 MB/s) - ‘train.txt’ saved [661664/661664]

--2024-01-24 15:23:35--  https://raw.githubusercontent.com/oya163/bert-llm/master/CyberSecurityNER/cyner_data/valid.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [3]:
from datasets import load_dataset

data_files = {
    "train": "train.txt",
    "validation": "valid.txt",
    "test": "test.txt",
}

raw_datasets = load_dataset("load_ner.py", data_files=data_files)

/home/e077926/miniconda3/Bib/envs/tata_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 53.04it/s]
Generating train split: 2811 examples [00:00, 7810.92 examples/s]
Generating validation split: 813 examples [00:00, 8647.79 examples/s]
Generating test split: 748 examples [00:00, 8086.16 examples/s]


In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2811
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 813
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 748
    })
})

In [5]:
raw_datasets["train"][0]

{'id': '0',
 'tokens': ['Super',
  'Mario',
  'Run',
  'Malware',
  '#',
  '2',
  '–',
  'DroidJack',
  'RAT',
  'Gamers',
  'love',
  'Mario',
  'and',
  'Pokemon',
  ',',
  'but',
  'so',
  'do',
  'malware',
  'authors',
  '.'],
 'ner_tags': [1, 2, 2, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0]}

In [29]:
raw_datasets["train"][100]

{'id': '100',
 'tokens': [']', 'com', 'hxxp', ':', '//mailsa-qaw', '[', '.'],
 'ner_tags': [8, 8, 7, 8, 8, 8, 8]}

# 01 Load my data

In [10]:
import pandas as pd

In [11]:
df=pd.read_csv('/home/e077926/buscode_2023/05_E2_tata/data/final_dataset.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,sentence_id,words,labels,ner_tags
0,0,tan,O,0
1,0,woon,O,0
2,0,yann,O,0
3,11,book,B-ORG,1
4,11,tak,O,0


In [39]:
df.tail()

,sentence_id,words,labels,ner_tags
66826,18086,cashier,O,0
66827,18114,cashier,O,0
66828,18114,cashier,O,0
66829,18138,rm,O,0
66830,18138,rm,O,0


## Split data

In [20]:
from sklearn.model_selection import train_test_split
train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, eval_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

In [41]:
# Grouper les données par sentence_id
train = train_df.groupby('sentence_id').apply(lambda x: {'id': str(x['sentence_id'].iloc[0]),
                                                          'tokens': list(x['words']),
                                                          'ner_tags': list(x['ner_tags'])}).to_dict()

test=test_df.groupby('sentence_id').apply(lambda x: {'id': str(x['sentence_id'].iloc[0]),
                                                          'tokens': list(x['words']),
                                                          'ner_tags': list(x['ner_tags'])}).to_dict()
evalu=eval_df.groupby('sentence_id').apply(lambda x: {'id': str(x['sentence_id'].iloc[0]),
                                                          'tokens': list(x['words']),
                                                          'ner_tags': list(x['ner_tags'])}).to_dict()


In [44]:
train[0]['tokens']

['woon', 'tan', 'yann']

## Display

In [12]:
label_names=df['labels'].unique().tolist()
label_names

['O', 'B-ORG', 'I-ORG', 'B-ADD', 'I-ADD', 'B-TIM', 'B-TOT', 'I-TOT', 'I-TIM']

In [74]:
train

{0: {'id': '0', 'tokens': ['woon', 'tan', 'yann'], 'ner_tags': [0, 0, 0]},
 1: {'id': '1',
  'tokens': ['25122018', 'date', '81339', 'pm'],
  'ner_tags': [5, 0, 0, 0]},
 2: {'id': '2', 'tokens': ['manis', 'cashier'], 'ner_tags': [0, 0]},
 3: {'id': '3', 'tokens': ['member'], 'ner_tags': [0]},
 5: {'id': '5', 'tokens': ['codedesc', 'amouit'], 'ner_tags': [0, 0]},
 6: {'id': '6', 'tokens': ['rm', 'quy', 'rm'], 'ner_tags': [0, 0, 0]},
 7: {'id': '7', 'tokens': ['fish', 'kiddy'], 'ner_tags': [0, 0]},
 8: {'id': '8', 'tokens': ['1pc', '600'], 'ner_tags': [0, 0]},
 9: {'id': '9', 'tokens': ['total'], 'ner_tags': [0]},
 10: {'id': '10',
  'tokens': ['adjustment', '000', 'rour'],
  'ner_tags': [0, 0, 0]},
 11: {'id': '11', 'tokens': ['book', 'bhd', 'taman'], 'ner_tags': [1, 2, 1]},
 12: {'id': '12', 'tokens': ['d', 'rm'], 'ner_tags': [0, 0]},
 13: {'id': '13', 'tokens': ['960'], 'ner_tags': [0]},
 14: {'id': '14', 'tokens': ['cash'], 'ner_tags': [0]},
 15: {'id': '15', 'tokens': ['change'], 'n

In [47]:
words = train[100]["tokens"]
labels = train[100]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

for x, y in zip(line1.split(), line2.split()):
    print(x, '\t', y)

total 	 O
8091 	 O
000 	 O


## Tokenization

In [48]:
from transformers import AutoTokenizer

model_checkpoint = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json: 100%|██████████| 616/616 [00:00<00:00, 3.68MB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:01<00:00, 4.49MB/s]
tokenizer.json: 100%|██████████| 9.10M/9.10M [00:03<00:00, 2.89MB/s]


In [75]:
# Supposons que train soit votre dictionnaire
tokens_list = [item['tokens'] for item in train.values()]
tokens_list

Liste de tous les tokens : [['woon', 'tan', 'yann'], ['25122018', 'date', '81339', 'pm'], ['manis', 'cashier'], ['member'], ['codedesc', 'amouit'], ['rm', 'quy', 'rm'], ['fish', 'kiddy'], ['1pc', '600'], ['total'], ['adjustment', '000', 'rour'], ['book', 'bhd', 'taman'], ['d', 'rm'], ['960'], ['cash'], ['change'], ['returnar', 'goods', 'sold'], ['exchangeable'], ['thank', 'you'], ['agaty', 'come', 'please'], ['b94', '7w'], ['no5', '59', 'jalan', 'sagu', '5557', '18'], ['81100', 'bahru'], ['lam', 'mcu'], ['tdo1167104', 'ho', 'document'], ['woon', 'yann'], ['mb'], ['room', 'of', 'no', '050100025279'], ['gty', 'amtrh', 'price'], ['stprivilege'], ['1000', '1'], ['lampstitch', 'gftable'], ['62483', '5590'], ['1000', 'nisc'], ['2', 'htotal'], ['total', 'aht'], ['home', 'indah', 'beco'], ['adj'], ['again', 'come', 'thank'], ['not', 'are', 'retui', 'goods', 'sold'], ['and', 'healing', 'wholesale'], ['13', 'jalan', '27'], ['jaya', 'tanan'], ['81100'], ['fax073558160'], ['01', '204959', '1910201

In [79]:
tag_list = [item['ner_tags'] for item in train.values()]
tag_list

[[0, 0, 0],
 [5, 0, 0, 0],
 [0, 0],
 [0],
 [0, 0],
 [0, 0, 0],
 [0, 0],
 [0, 0],
 [0],
 [0, 0, 0],
 [1, 2, 1],
 [0, 0],
 [0],
 [0],
 [0],
 [0, 0, 0],
 [0],
 [0, 0],
 [0, 0, 0],
 [0, 0],
 [0, 4, 4, 4, 3, 4],
 [4, 4],
 [0, 0],
 [0, 0, 0],
 [0, 0],
 [0],
 [0, 0, 0, 0],
 [0, 0, 0],
 [0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [2, 1, 0],
 [0],
 [0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0],
 [4, 4, 3],
 [3, 0],
 [4],
 [0],
 [0, 0, 5],
 [0, 0, 0, 0],
 [0, 0],
 [0],
 [0, 0],
 [0, 0, 0, 0, 0],
 [0],
 [0, 0],
 [0, 0, 0],
 [0, 0],
 [0, 0, 0],
 [0, 0],
 [1, 2, 0],
 [0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0],
 [0],
 [0, 0, 0],
 [0, 0, 0, 0],
 [0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [4, 3, 4, 4],
 [4, 4],
 [4, 4, 4, 4],
 [4, 4, 4],
 [0],
 [0, 0, 0, 0],
 [0, 0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0],
 [0, 0],
 [0, 0, 0, 0, 0],
 [0],
 [0, 0, 0],
 [2],
 [0, 0],
 [0, 0, 0],
 [0],
 [0, 0],
 [0],
 [0],
 [0],
 [0],
 [0, 0, 0],
 [0, 0],
 [4, 4, 4],
 [4, 4]

In [80]:
inputs = tokenizer(tokens_list, is_split_into_words=True)
print(inputs.tokens())

TypeError: PreTokenizedEncodeInput must be Union[PreTokenizedInputSequence, Tuple[PreTokenizedInputSequence, PreTokenizedInputSequence]]

In [57]:
print(inputs.word_ids())

[None, 0, 1, 2, 2, None]


# Data Preprocessing

In [58]:
#  Align the number of labels and the tokens
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [71]:
labels = train[0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0]
[-100, 0, 0, 0, 0, -100]


In [72]:
# Helper function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [66]:
# Supposons que train soit un dictionnaire avec des données
train_keys = train.keys()

# Ou si train est une liste d'éléments, prenez le premier élément et accédez à ses attributs ou clés
if train:
    first_element = train[0]
    
    # Si c'est un dictionnaire
    if isinstance(first_element, dict):
        train_keys = first_element.keys()
    
    # Si c'est une liste ou un tableau
    elif isinstance(first_element, (list, np.ndarray)):
        # Supposons que le premier élément est une ligne de données
        train_keys = range(len(first_element))

# Afficher les noms des colonnes
print("Noms des colonnes :", train_keys)


Noms des colonnes : dict_keys(['id', 'tokens', 'ner_tags'])


In [68]:
# Tokenize all the examples from the datasets
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_keys,
)

Map:   0%|          | 0/2811 [00:00<?, ? examples/s]

Map: 100%|██████████| 748/748 [00:00<00:00, 13245.15 examples/s]


# Fine Tuning

## Data Collation

In [81]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Setup Evaluation

In [83]:
import evaluate
import numpy as np

metric = evaluate.load("seqeval")


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [84]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [85]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:  72%|███████▏  | 1.61G/2.24G [02:21<00:41, 15.3MB/s]Error while downloading from https://cdn-lfs.huggingface.co/xlm-roberta-large/2dfa19f172412917cab174da04b46e2134811b723666965fd0aabd97caa6e23b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1706369397&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNjM2OTM5N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby94bG0tcm9iZXJ0YS1sYXJnZS8yZGZhMTlmMTcyNDEyOTE3Y2FiMTc0ZGEwNGI0NmUyMTM0ODExYjcyMzY2Njk2NWZkMGFhYmQ5N2NhYTZlMjNiP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=Wi1Ioi0SqybyI4vXTP4T7Od%7EiynCp77NhKAS0SmbmcJR9mF2zfkF%7EZ5PpNEYAJIfHdK3wvcrCgM4%7ElxdjhFAUlb8pfWweDJEyVTFublE4KIKsDMADeyJG0o7iwpifUevaPLu%7ER1y2U1nRsZ9PE2FUA7mtXou5pNC8ZAXnRpfqkOneYdCAjD3prZ4HtDxXU5UkitStY77BQby3RW91pVwXV7IjjmX8zlCagVsq-ZFNgaI0afKmKE-es6ZTgNer-7c%7E1Liu6Ibd2qY40BSkRR5CJd4NCgGufBiKfcuKfELzWKNex

In [86]:
model.config.num_labels

9

# Training

In [3]:
tokenized_datasets = train.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_keys,
)

NameError: name 'train' is not defined

In [2]:
tokenized_datasets

NameError: name 'tokenized_datasets' is not defined

In [88]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

https://github.com/sawantbhardwaj/Custom-NER-with-BERT-on-MIT-Movie-Corpus-/blob/main/BERT_NER.ipynb

In [4]:

from simpletransformers.ner import NERModel , NERArgs

/home/e077926/miniconda3/Bib/envs/tata_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-24 16:41:12.834184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 16:41:12.834242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 16:41:12.888708: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 16:41:13.004478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Te

In [5]:

model_args = NERArgs()
model_args.num_train_epochs = 2
model_args.learning_rate = 1e-4
model_args.overwrite_output_dir =True
model_args.train_batch_size = 32
model_args.eval_batch_size = 8
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 1000
model_args.evaluate_during_training_verbose = True

In [13]:
model_bert = NERModel('bert', 'bert-base-cased', labels=label_names, args = model_args)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model_distilbert = NERModel('distilbert', 'distilbert-base-cased', labels=label_names, args = model_args)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
df

,sentence_id,words,labels,ner_tags
0,0,tan,O,0
1,0,woon,O,0
2,0,yann,O,0
3,11,book,B-ORG,1
4,11,tak,O,0
...,...,...,...,...
66826,18086,cashier,O,0
66827,18114,cashier,O,0
66828,18114,cashier,O,0
66829,18138,rm,O,0


In [22]:
train_df.head()

,sentence_id,words,labels,ner_tags
58217,16419,cashier,O,0
25829,7625,rm,O,0
40515,11442,9068722,O,0
27558,8190,g30,I-ADD,4
51854,14636,amountrm,O,0


In [ ]:
# Supposons que train soit votre dictionnaire
tokens_list = [item['tokens'] for item in train.values() if isinstance(item['tokens'], list)]

# Afficher tous les tokens
print("Liste de tous les tokens :", tokens_list)


In [17]:
model_distilbert.train_model(train_df,eval_data = test_df)

  0%|          | 0/21 [00:00<?, ?it/s]


TypeError: 'float' object is not iterable